# Análisis de Transacciones y Detección de Alertas Sospechosas

**1. Introducción**

Este análisis tiene como objetivo examinar un conjunto de datos sobre transacciones financieras, prestando especial atención a la variable *flagged_alert*, que señala si una transacción ha sido considerada sospechosa (1) o no sospechosa (0). La idea es identificar posibles falsos positivos y evaluar cómo el monto (amount) y el tipo de transacción (transaction_type) influyen en la generación de alertas.

 *📌 Nota:Este análisis se realizó sobre un dataset ficticio con fines educativos y analíticos. No se está manejando información real ni confidencial.*

**2. Descripción General del Dataset**

Se utilizó el archivo transaction_data.csv que contiene 5000 transacciones. La variable principal de interés es el monto de la transacción *(amount)*, acompañado de indicadores como el tipo de transacción *(transaction_type)* y si fue marcada como sospechosa *(flagged_alert)*.

   Estadísticas del Monto
   - Monto mínimo: 10.06
   - Monto máximo: 4997.79
   - Media: 2474.55
   - Desviación estándar: 1435.00
   - Rango intercuartílico:
      - 25% ≤ 1236.74
      - 75% ≤ 3710.84
   Debido a la desviación estándar y la media observamos que hay una alta variabilidad en los montos.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

f = pd.read_csv("transaction_data.csv")
print(f.describe())


**Análisis de Alertas Sospechosas**

Observamos que :
- El *68.28%* de las transacciones fueron marcadas como *no sospechosas*.
- El *31.72%* de las transacciones fueron marcadads como *sospechosas* . 

Esto nos indica que hay una cantidad considerable de *'alertas'* , esto lo confirmamos mediante un gráfico de barras donde se observa un mayor número de transacciones no marcadas como sospechosas.


In [ ]:

print(f['flagged_alert'].value_counts(normalize=True))
f['flagged_alert'].value_counts().plot(kind='bar', figsize=(10, 6),color='salmon')
plt.xlabel('Flagged Alert')
plt.ylabel('Cantidad de Transacciones')
plt.title('Detección de falsos positivos')
plt.show()


**Comparación de los montos según alertas**

Un análisis de caja nos reveló lo siguiente
- la mediana de los montos en transacciones sospechosas es mayor que la de las no sospechosas.
- Tienen rango de valores similares y alcanzan montos maximos parecidos.

Con lo anterior podemos afirmar que
- Se tiendne a marcar montos más altos como sospechosos
- Hay transacciones grandes que pueden ser no marcadas como sospechosas y algunas pequeñas que si lo son

Entonces si hay muchas transacciones con montos bajos marcados como transacción sospechosa podría ser señal de Falsos Positivos



In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(x=f['flagged_alert'],y=f['amount'],color='salmon')
plt.xlabel('Flagged Alert')
plt.ylabel('Monto de la Transacción')
plt.title('Distribución de Montos en Transacciones Marcadas y No Marcadas')
plt.show()

**Posibles Falsos Positivos (FP)**

Para identificar posibles falsos positivos,inicialmente se analizaron transacciones marcadas como sospechosas con montos inferiores a la mediana (2474.55) , es decir, se respondió a la pregunta *¿Cuantás transacciones de bajo monto fueron marcadas como sospechosas?*.

Se obtuvieron 547 transacciones sospechosas con montos bajos,lo cuál nos sugiere que se podría estar marcando incorrectamente algunas transacciones normales.

Ahora nos hacemos la pregunta, *¿Solo se detectan montos altos?*

Al analizar las estadísticas de los montos según la clasificación de *alerta*, observamos lo siguiente:
- Las transacciones de mayor monto tienen más probabilidades de ser marcadas como sospechosas.
- Sin embargo, también hay transacciones pequeñas que fueron clasificadas como sospechosas.
- El mínimo de ambas categorías es similar (10.06 y 10.15), lo cuál nos indica que existen montos bajos dentro de las transacciones sospechosas.
- El 25% de las transacciones sospechosas tienen un monto igual o inferior a 1720.60, lo cual confirma que no todas las transacciones marcadas como sospechosas son de montos altos.

Esto nos sugiere que, *aunque el monto es un factor clave en la clasificación de alertas, existen otros criterios que pueden llevar a que transacciones de bajo monto sean señaladas como sospechosas*.

Al comparar la distribución de los montos para transacciones sospechosas y no sospechosas mediante un Análisis de Distribución (KDE Plot) se observa una diferencia notable en la forma de las curvas.Esto nos confirma que el monto tiene un impacto importante en la probabilidad de que una transacción sea marcada como sospechosa.

Al finalizar se analizaron los *montos por tipo de transacción*,observamos lo siguiente

- Algunos tipos de transacción presentan mayores tasas de sospecha que otros y se presentan variaciones en el monto promedio según el tipo de transacción.
- Esto nos llega a indicar que el tipo de transacción también influye en la clasificación y podría ser una fuente de falsos positivos.


In [ ]:
transacciones_bajo_monto = f[(f['amount'] < f['amount'].median()) & (f['flagged_alert'] == 1)]
print(f"Cantidad de transacciones sospechosas con monto bajo: {len(transacciones_bajo_monto)}")
#si hay muchas transacciones con montos bajos con f['flagged_alert'] == 1 podría ser señal de FP

#solo se detectan montos bajos?
print(f.groupby('flagged_alert')['amount'].describe())

#el monto tiene un impacto en las alertas?
sns.kdeplot(f[f['flagged_alert'] == 0]['amount'], label="No sospechosa", fill=True)
sns.kdeplot(f[f['flagged_alert'] == 1]['amount'], label="Sospechosa", fill=True)
plt.xlabel("Monto de la Transacción")
plt.ylabel("Densidad")
plt.title("Distribución de Montos en Transacciones Marcadas y No Marcadas")
plt.legend()
plt.show()


# Contamos alertas por tipo de transacción
print(f.groupby("transaction_type")["flagged_alert"].mean() * 100)
print(f.groupby("transaction_type")["amount"].mean())

**Conclusiones**
- El monto de la transacción influye significativamente en la clasificación de alertas.
- Existen indicios de falsos positivos, especialmente en montos bajos.
- El tipo de transacción también parece tener un impacto en la probabilidad de ser marcada como sospechosa.
- Se recomienda un análisis más profundo para ajustar los parámetros del sistema de detección y reducir errores de clasificación.

*📌 Nota final: Este análisis se basa en datos simulados y no representa información financiera real.*